# HW4 - Pandas exercises

We open with some imports as usual

In [ ]:
import urllib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Some helpful functions 
just use these as be and don't worry too much about about them for now, we will learn how they work soon

In [87]:
# Same data manipulation to get USGS streamflow as
# a pandas dataframe as before
def create_usgs_url(site_no, begin_date, end_date):
    return (
        f'https://waterdata.usgs.gov/nwis/dv?'
        f'cb_00060=on&format=rdb&referred_module=sw&'
        f'site_no={site_no}&'
        f'begin_date={begin_date}&'
        f'end_date={end_date}'
    )

def open_usgs_data(site, begin_date, end_date):
    url = create_usgs_url((site), begin_date, end_date)
    response = urllib.request.urlopen(url)
    df = pd.read_table(
        response,
        comment='#',
        skipfooter=1,
        sep='\s+',
        names=['agency', 'site', 'date', 'streamflow (ft^3/s)', 'quality_flag'],
        index_col=2,
        #parse_dates=True,
        engine='python'
    ).iloc[2:]

    # Now convert the streamflow data to floats and
    # the index to datetimes. When processing raw data
    # it's common to have to do some extra postprocessing
    df['streamflow (ft^3/s)'] = df['streamflow (ft^3/s)'].astype(np.float64)
    df.index = pd.DatetimeIndex(df.index)
    return df

def open_daymet_data(lat, lon, begin_date, end_date):
    args = {'lat':  lat, 'lon': lon, 'format': 'csv',
            'start': begin_date, 'end': end_date}
    query = urllib.parse.urlencode(args)
    url = f"https://daymet.ornl.gov/single-pixel/api/data?{query}"
    response = urllib.request.urlopen(url)
    df = pd.read_csv(response, header=6)
    datestring = (df['year'].astype(str) + df['yday'].astype(str))
    dates = pd.to_datetime(datestring, format='%Y%j')
    df.index = pd.DatetimeIndex(dates)
    return df

# Additionally, setting some constants

In [88]:
site = '09506000'
begin_date = '1992-09-25'
end_date = '2022-09-25'
lat = 34.4483605
lon = -111.7898705

# Now open up some datasets from the above commands, you are almost to your exercises

In [91]:
verde_df = open_daymet_data(lat, lon, begin_date, end_date)
usgs_df = open_usgs_data(site, begin_date, end_date)
verde_df = verde_df.reindex(verde_df.index)
verde_df['streamflow (ft^3/s)'] = usgs_df['streamflow (ft^3/s)']
verde_df.head()

,year,yday,dayl (s),prcp (mm/day),srad (W/m^2),swe (kg/m^2),tmax (deg c),tmin (deg c),vp (Pa),streamflow (ft^3/s)
1992-09-25,1992,269,42469.79,0.0,413.81,0.0,33.57,12.92,1489.09,129.0
1992-09-26,1992,270,42337.25,0.0,419.69,0.0,34.03,12.61,1459.42,121.0
1992-09-27,1992,271,42204.86,0.0,436.54,0.0,34.89,11.41,1348.48,122.0
1992-09-28,1992,272,42072.64,0.0,421.55,0.0,35.44,12.82,1479.16,127.0
1992-09-29,1992,273,41940.63,0.0,380.67,0.0,33.42,14.18,1616.07,120.0


# 1. How do you get a listing of the columns in `verde_df`?

In [5]:
# TODO: your code here

# 2. How do you select the streamflow column in `verde_df`?

In [6]:
# TODO: your code here

# 3. How do you plot the streamflow in `verde_df`?

In [7]:
# TODO: your code here

# 4. What is the mean streamflow value for the 30 year period?

In [8]:
# TODO: your code here

# 5. What is the maximum value for the 30 year period?


In [9]:
# TODO: your code here

# 6. How do you find the maximum streamflow value for each year?

In [10]:
# TODO: your code here

# 7. How do you make a scatter plot of `dayl (s)` versus `tmax (deg c)`?
#### INFO: `dayl` is the day length in seconds and `tmax` is the daily maximum temperature

In [11]:
# TODO: your code here

# 8. How do you calculate (and plot) the mean daily minimum temperature for each day of year?  And plot it?
#### INFO: Daily minimum temperature is in the column `tmin (deg c)`

In [12]:
# TODO: your code here

## 9. Building a streamlined workflow

For the previous exercises you may have noticed I gave you both the site ID and the lat/lon. Ideally, we should be able to just give you the site ID and get the lat/lon from the USGS site. This will be your task for this exercise. You can retrieve site metadata from the USGS from the following site:

https://waterdata.usgs.gov/nwis/inventory?search_criteria=search_site_no&submitted_form=introduction

Use this tool to retrieve the tabular data for the Verde river at the site 09506000. You will need to select 3 columns:

- Decimal Latitude
- Decimal Longitude
- Drainage Area (returned in square miles)

You will also need to toggle the "Site-description information displayed in" radio button and select "tab-separated format" from the drop down. Once you have retrieved the table, use it to build a function that lets you put in any site ID and get the lat/lon and drainage area.

Then, use this function to get the lat/lon and drainage area for the site 09498500 (SALT RIVER NEAR ROOSEVELT, AZ).

In [ ]:
def get_usgs_site_info(site_id):
    url = f""                                     # TODO: Change this line 
    response = urllib.request.urlopen(url)
    df = pd.read_table(       
        response,
        comment='#',
        skiprows=[25],
        sep='\s+',
        engine='python'
    )

    # Pull out the data
    lat = df['dec_lat_va'].item()
    lon = df['dec_long_va'].item()
    area = df['drain_area_va'].item()

    return lat, lon, area

In [82]:
site_id = '09498500'
lat, lon, area = get_usgs_site_info(site_id)

## 10. Calculating the Runoff ratio

From the site info, open the USGS streamflow and Daymet data using the previously defined functions for the period 1990-01-01 to 2020-01-01. Calculate the yearly total (sum) precipitation and streamflow values. From this, calculate the yearly runoff ratio, which is simply the ratio of the total runoff to total precipitation. You will need to convert the streamflow from cubic feet per second to mm of flow. Use the given conversion factors to do these conversions. Finally, make a histogram plot showing the distribution of runoff ratios over the full period.

**Note**: The Daymet data is extracted from a single pixel from a 1x1 km grid, but for our purposes assume this is the total over the entire basin. 

In [ ]:
seconds_per_day = 86400
m3_per_ft3 = 0.0283
m2_per_km2 = 1e6
mm_per_m = 1000
km2_per_mi2 = 2.59

# TODO: Your code here